In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.callbacks import StreamingStdOutCallbackHandler

# Caching for embeddings
cache_dir = LocalFileStore("./.cache/")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_length=20,
    max_token_limit=100,
    memory_key="chat_history",
    return_messages=True,
)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# Loading and splitting the document
loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)


# Setting up embeddings with caching
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# Creating the vector store and retriever
vector_store = FAISS.from_documents(docs, cached_embeddings)
retriver = vector_store.as_retriever()



prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | chat
)

chain.invoke("Describe Victory Mansions") 

Victory Mansions is a place where Winston lives. It is described as a dilapidated apartment building with shabby conditions. The building is run-down, with faulty plumbing, intermittent electricity, and a general sense of neglect. The atmosphere is grim and oppressive, reflecting the overall bleakness of the society in which Winston resides.

AIMessageChunk(content='Victory Mansions is a place where Winston lives. It is described as a dilapidated apartment building with shabby conditions. The building is run-down, with faulty plumbing, intermittent electricity, and a general sense of neglect. The atmosphere is grim and oppressive, reflecting the overall bleakness of the society in which Winston resides.')

In [4]:
chain.invoke("Is Aaronson guilty?")

According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.

AIMessageChunk(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [5]:
chain.invoke("What message did he write in the table?")

He wrote "2+2=5" in the dust on the table.

AIMessageChunk(content='He wrote "2+2=5" in the dust on the table.')

In [6]:
chain.invoke("Who is Julia?")

Julia is a character mentioned in the text. She is someone who the main character has strong feelings for and experiences a moment of overwhelming love and concern for.

AIMessageChunk(content='Julia is a character mentioned in the text. She is someone who the main character has strong feelings for and experiences a moment of overwhelming love and concern for.')